<div style="float:right; padding-top: 15px; padding-right: 15px">
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="250">
        </a>
    </div>
</div>

# Data Polishing 3 webs

## 0. python imports

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy

from hdbscan import HDBSCAN
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.lang.es import Spanish
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle
from umap import UMAP

with warnings.catch_warnings():
    warnings.simplefilter('ignore')

## 1. data loading, merging and cleaning by site

### Gamereactor

In [2]:
gamereactor_df1 = pd.read_csv('../data/gamereactor_100l.csv')

In [3]:
gamereactor_df2 = pd.read_csv('../data/gamereactor_100_500l.csv')

In [4]:
gamereactor_df3 = pd.read_csv('../data/gamereactor_500_1249l.csv')

In [5]:
gamereactor_df4 = pd.read_csv('../data/gamereactor_1250_last.csv')

In [6]:
gamereactor_data = [gamereactor_df1, gamereactor_df2, gamereactor_df3, gamereactor_df4]
gamereactor_df = pd.concat(gamereactor_data)

In [7]:
gamereactor_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2260 entries, 0 to 1012
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   site      2260 non-null   object 
 1   url_link  2260 non-null   object 
 2   author    2260 non-null   object 
 3   game      2260 non-null   object 
 4   company   2260 non-null   object 
 5   genre     2260 non-null   object 
 6   platform  2260 non-null   object 
 7   text      2260 non-null   object 
 8   score     2260 non-null   float64
dtypes: float64(1), object(8)
memory usage: 176.6+ KB


In [8]:
gamereactor_df.describe()

,score
count,2260.000000
mean,7.430088
std,1.348512
min,1.000000
25%,7.000000
50%,8.000000
75%,8.000000
max,10.000000


In [9]:
gamereactor_df = gamereactor_df.reset_index(drop=True)

In [10]:
# Duplicates

gamereactor_df.duplicated().value_counts()# - False 2260

False    2260
dtype: int64

In [12]:
#gamereactor_df.to_csv('../data/gamereactor_all.csv', index=False)

In [8]:
# Testing empties per column

# (gamereactor_df['game'] == 'None').value_counts() - False 1247
# (gamereactor_df['site'] == 'None').value_counts() - False 1247
# (gamereactor_df['author'] == 'None').value_counts() - False 1247
# (gamereactor_df['url_link'] == 'None').value_counts() - False 1247
# (gamereactor_df['company'] == 'None').value_counts() - False 1122 / True 125
# (gamereactor_df['genre'] == 'None').value_counts() - False 1233 / True 14
# (gamereactor_df['platform'] == 'None').value_counts() - False 1230 / True 17
# (gamereactor_df['score'] == 'None').value_counts() - False 1247


In [23]:
gamereactor_df.head(3)

,site,url_link,author,game,company,genre,platform,text,score
0,Gamereactor,https://www.gamereactor.es/squad-analisis/?sid...,Mike Holmes,Squad,None,Acción,PC,Anda que no ha llovido desde que jugamos Squad...,8.0
1,Gamereactor,https://www.gamereactor.es/super-mario-bros-35...,Sergio Figueroa,Super Mario Bros. 35 - Battle Royale,Nintendo,Plataformas,Nintendo Switch,"No se habían olvidado de él, lo que pasaba es ...",7.0
2,Gamereactor,https://www.gamereactor.es/crash-bandicoot-4-i...,Eirik Hyldbakk Furu,Crash Bandicoot 4: It's About Time,Activision,Plataformas,"PS4, Xbox One",Aunque creo que Ford se pasó un poco con la ca...,8.0


*Observations:*
- Many Company nones (10%)
- Few Genre and Platform nones (1%/2%)

### Meristation

In [15]:
meristation_df1 = pd.read_csv('../data/meristation_50p.csv')

In [16]:
meristation_df2 = pd.read_csv('../data/meristation_51_150p.csv')

In [17]:
meristation_data = [meristation_df1, meristation_df2]
meristation_df = pd.concat(meristation_data)

In [18]:
meristation_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4427 entries, 0 to 2985
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   site      4427 non-null   object 
 1   url_link  4427 non-null   object 
 2   author    4424 non-null   object 
 3   game      4427 non-null   object 
 4   company   4427 non-null   object 
 5   genre     4427 non-null   object 
 6   platform  4427 non-null   object 
 7   text      4427 non-null   object 
 8   score     4427 non-null   float64
dtypes: float64(1), object(8)
memory usage: 345.9+ KB


In [19]:
meristation_df.describe()

,score
count,4427.000000
mean,7.446307
std,1.234652
min,0.500000
25%,7.000000
50%,7.500000
75%,8.300000
max,10.000000


In [20]:
# Duplicates

meristation_df.duplicated().value_counts()# - False 4427

False    4427
dtype: int64

In [21]:
meristation_df.to_csv('../data/meristation_all.csv', index=False)

In [39]:
# Testing empties per column

# (meristation_df['game'] == 'None').value_counts()# - False 1441
# (meristation_df['site'] == 'None').value_counts()# - False 1441
# (meristation_df['author'] == 'None').value_counts()# - False 1308 / True 133
# (meristation_df['url_link'] == 'None').value_counts()# - False 1441
# (meristation_df['company'] == 'None').value_counts()# - False 1278 / True 163
# (meristation_df['genre'] == 'None').value_counts()# - False 1429 / True 12
# (meristation_df['platform'] == 'None').value_counts()# - False 1441
# (meristation_df['score'] == 'None').value_counts()# - False 1441

In [22]:
meristation_df.head(3)

,site,url_link,author,game,company,genre,platform,text,score
0,meristation,https://as.com/meristation/2020/10/05/analisis...,Carlos Forcada,OkunoKA Madness,Ignition Entertainment,Plataformas,XBO NSW PS4 PC,"\n\n Aunque parece que están ahí casi siempre,...",7.4
1,meristation,https://as.com/meristation/2020/10/03/analisis...,Cristian Ciuraneta,art of rally,Funselektor,Conducción,PC,Los fans de los videojuegos de carreras están ...,7.5
2,meristation,https://as.com/meristation/2020/10/01/analisis...,Sergio C. González\nSergio5Glez,Crash Bandicoot 4: It's About Time,Activision,Plataformas,PS4 XBO,\n\n Crash Bandicoot N. Sane Trilogy fue toda ...,8.3


*Observations:*
- Many Author and Company nones (10%)
- Few Genre (1%)
- Platform names poor labeling

### Revogamers

In [24]:
revogamers_df1 = pd.read_csv('../data/revogamers_100l.csv')

In [25]:
revogamers_df2 = pd.read_csv('../data/revogamers_100_500l.csv')

In [26]:
revogamers_df3 = pd.read_csv('../data/revogamers_500_999l.csv')

In [27]:
revogamers_df4 = pd.read_csv('../data/revogamers_1000_1257l.csv')

In [28]:
revogamers_data = [revogamers_df1, revogamers_df2, revogamers_df3, revogamers_df4]
revogamers_df = pd.concat(revogamers_data)

In [29]:
revogamers_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1120 entries, 0 to 256
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   site      1120 non-null   object 
 1   url_link  1120 non-null   object 
 2   author    1120 non-null   object 
 3   game      1120 non-null   object 
 4   company   1120 non-null   object 
 5   genre     1120 non-null   object 
 6   platform  1120 non-null   object 
 7   text      1119 non-null   object 
 8   score     1120 non-null   float64
dtypes: float64(1), object(8)
memory usage: 87.5+ KB


In [30]:
revogamers_df.describe()

,score
count,1120.000000
mean,7.197321
std,1.277898
min,0.000000
25%,6.500000
50%,7.500000
75%,8.000000
max,10.000000


In [31]:
revogamers_df = revogamers_df.reset_index(drop=True)

In [32]:
# Duplicates

revogamers_df.duplicated().value_counts()# - False 1120

False    1120
dtype: int64

In [48]:
#revogamers_df.to_csv('../data/revogamers_all.csv', index=False)

In [59]:
# Testing empties per column

# (revogamers_df['game'] == 'None').value_counts()# - False 863
# (revogamers_df['site'] == 'None').value_counts()# - False 863
# (revogamers_df['author'] == 'None').value_counts()# - False 863
# (revogamers_df['url_link'] == 'None').value_counts()# - False 863
# (revogamers_df['company'] == 'None').value_counts()# - False 402 / True 461
# (revogamers_df['genre'] == 'None').value_counts()# - False 406 / True 457
# (revogamers_df['platform'] == 'None').value_counts()# - False 863
# (revogamers_df['score'] == 'None').value_counts()# - False 863


In [33]:
revogamers_df.head(3)

,site,url_link,author,game,company,genre,platform,text,score
0,revogamers,https://www.revogamers.net/analisis-w/analisis...,Carlos Firás,Going Under,None,None,Nintendo Switch,Llega a nuestras Nintendo Switch un juego que ...,8.0
1,revogamers,https://www.revogamers.net/analisis-w/analisis...,Javier Aranda,Kirby Fighters 2,HAL Laboratory,Lucha,Nintendo Switch,Kirby ha demostrado en más de una ocasión que ...,7.5
2,revogamers,https://www.revogamers.net/analisis-w/analisis...,Carlos Firás,Lost Ember,None,None,Nintendo Switch,Llega a nuestras Nintendo Switch un juego que ...,7.5


*Observations:*
- Too much Genre and Company nones (+50%)
- One null text

### Vandal

In [35]:
vandal_df1 = pd.read_csv('../data/vandal_2225l.csv')

In [36]:
vandal_df2 = pd.read_csv('../data/vandal_2225_4482l.csv')

In [37]:
vandal_data = [vandal_df1, vandal_df2]
vandal_df = pd.concat(vandal_data)

In [39]:
vandal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4471 entries, 0 to 2256
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   site      4471 non-null   object 
 1   url_link  4471 non-null   object 
 2   author    4471 non-null   object 
 3   game      4471 non-null   object 
 4   company   4453 non-null   object 
 5   genre     4469 non-null   object 
 6   platform  4471 non-null   object 
 7   text      4435 non-null   object 
 8   score     4471 non-null   float64
dtypes: float64(1), object(8)
memory usage: 349.3+ KB


In [40]:
vandal_df.describe()

,score
count,4471.000000
mean,7.611877
std,1.137457
min,1.000000
25%,7.000000
50%,7.800000
75%,8.500000
max,10.000000


In [41]:
vandal_df = vandal_df.reset_index(drop=True)

In [46]:
# Duplicates

#vandal_df.duplicated().value_counts()# - False 4356 (more that 100 duplicates, proceed to drop)

#vandal_df.drop_duplicates(inplace=True)

#len(vandal_df) - 4356

In [49]:
#vandal_df.to_csv('../data/vandal_all.csv', index=False)

In [ ]:
'''# Testing empties per column

# (revogamers_df['game'] == 'None').value_counts()# - False 863
# (revogamers_df['site'] == 'None').value_counts()# - False 863
# (revogamers_df['author'] == 'None').value_counts()# - False 863
# (revogamers_df['url_link'] == 'None').value_counts()# - False 863
# (revogamers_df['company'] == 'None').value_counts()# - False 402 / True 461
# (revogamers_df['genre'] == 'None').value_counts()# - False 406 / True 457
# (revogamers_df['platform'] == 'None').value_counts()# - False 863
# (revogamers_df['score'] == 'None').value_counts()# - False 863'''

In [47]:
vandal_df.head(3)

,site,url_link,author,game,company,genre,platform,text,score
0,Vandal,https://vandal.elespanol.com/analisis/ps4/budg...,Juan Rubio,Budget Cuts - Análisis,Neat Corporation,Acción Realidad Virtual,PS4 PC,Sabemos que empezamos a sonar como un disco ra...,8.0
1,Vandal,https://vandal.elespanol.com/analisis/pc/the-s...,Manu Delgado,The Survivalists - Análisis,Team17 Digital Ltd,Construcción Supervivencia,PC Xbox One Switch PS4,Desde su lanzamiento en 2015 la franquicia The...,8.0
2,Vandal,https://vandal.elespanol.com/analisis/switch/p...,Sergio Martín,Prinny 1 2: Exploded and Reloaded - Análisis,NIS America,Hack and Slash Plataformas 2D,Switch,NIS es una de esas compañías que entre sus num...,8.2


### 3DJuegos

In [50]:
tdjuegos_df1 = pd.read_csv('../data/tdjuegos_200l.csv')

In [51]:
tdjuegos_df2 = pd.read_csv('../data/tdjuegos_200_1400l.csv')

In [55]:
tdjuegos_df3 = pd.read_csv('../data/tdjuegos_1400_3000l.csv')

In [56]:
tdjuegos_df4 = pd.read_csv('../data/tdjuegos_3000_4470l.csv')

In [57]:
tdjuegos_data = [tdjuegos_df1, tdjuegos_df2, tdjuegos_df3, tdjuegos_df4]
tdjuegos_df = pd.concat(tdjuegos_data)

In [58]:
tdjuegos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4464 entries, 0 to 1469
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   site      4464 non-null   object 
 1   url_link  4464 non-null   object 
 2   author    4464 non-null   object 
 3   game      4464 non-null   object 
 4   company   4464 non-null   object 
 5   genre     4464 non-null   object 
 6   platform  4464 non-null   object 
 7   text      4464 non-null   object 
 8   score     4464 non-null   float64
dtypes: float64(1), object(8)
memory usage: 348.8+ KB


In [59]:
tdjuegos_df.describe()

,score
count,4464.000000
mean,7.688015
std,1.070927
min,2.000000
25%,7.000000
50%,8.000000
75%,8.500000
max,10.000000


In [60]:
tdjuegos_df = tdjuegos_df.reset_index(drop=True)

In [66]:
# Duplicates

#tdjuegos_df.duplicated().value_counts()# - False 4463 (1 duplicate, proceed to drop. There are more comparing game)

#tdjuegos_df.drop_duplicates(subset=['game'], inplace=True)

#len(tdjuegos_df)# - 3447

3447

In [67]:
#tdjuegos_df.to_csv('../data/tdjuegos_all.csv', index=False)

In [ ]:
'''# Testing empties per column

# (revogamers_df['game'] == 'None').value_counts()# - False 863
# (revogamers_df['site'] == 'None').value_counts()# - False 863
# (revogamers_df['author'] == 'None').value_counts()# - False 863
# (revogamers_df['url_link'] == 'None').value_counts()# - False 863
# (revogamers_df['company'] == 'None').value_counts()# - False 402 / True 461
# (revogamers_df['genre'] == 'None').value_counts()# - False 406 / True 457
# (revogamers_df['platform'] == 'None').value_counts()# - False 863
# (revogamers_df['score'] == 'None').value_counts()# - False 863'''

In [68]:
tdjuegos_df.head(3)

,site,url_link,author,game,company,genre,platform,text,score
0,3D Juegos,https://www.3djuegos.com/juegos/analisis/33227...,Adrián Suárez,Final Fantasy Crystal Chronicles Remastered Ed...,None,None,PS4 Switch Android iOS,Final Fantasy Crystal Chronicles Remastered Ed...,5.0
1,3D Juegos,https://www.3djuegos.com/juegos/analisis/36026...,Toni Piedrabuena,Captain Tsubasa: Rise of New Champions - Análisis,None,None,PC PS4 Switch,La obra de Yoichi Takahashi regresa al mundo d...,7.0
2,3D Juegos,https://www.3djuegos.com/juegos/analisis/22492...,Alberto Pastor,Wasteland 3 - Análisis,None,None,PC PS4 XOne Mac Linux,"En un mundo asolado por la guerra nuclear, rod...",9.0


## 2. Full data merge

In [2]:
gamereactor_df = pd.read_csv('../data/gamereactor_all.csv')
meristation_df = pd.read_csv('../data/meristation_all.csv')
revogamers_df = pd.read_csv('../data/revogamers_all.csv')
tdjuegos_df = pd.read_csv('../data/tdjuegos_all.csv')
vandal_df = pd.read_csv('../data/vandal_all.csv')

In [3]:
sites_dataframes = [gamereactor_df, meristation_df, revogamers_df, vandal_df, tdjuegos_df]

all_sites = pd.concat(sites_dataframes)

In [4]:
all_sites.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15610 entries, 0 to 3446
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   site      15610 non-null  object 
 1   url_link  15610 non-null  object 
 2   author    15607 non-null  object 
 3   game      15610 non-null  object 
 4   company   15593 non-null  object 
 5   genre     15608 non-null  object 
 6   platform  15610 non-null  object 
 7   text      15573 non-null  object 
 8   score     15610 non-null  float64
dtypes: float64(1), object(8)
memory usage: 1.2+ MB


gr_meri_revo.describe()

In [5]:
all_sites.dropna(inplace=True)

In [6]:
all_sites.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15551 entries, 0 to 3446
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   site      15551 non-null  object 
 1   url_link  15551 non-null  object 
 2   author    15551 non-null  object 
 3   game      15551 non-null  object 
 4   company   15551 non-null  object 
 5   genre     15551 non-null  object 
 6   platform  15551 non-null  object 
 7   text      15551 non-null  object 
 8   score     15551 non-null  float64
dtypes: float64(1), object(8)
memory usage: 1.2+ MB


In [7]:
all_sites = shuffle(all_sites)
all_sites.reset_index(drop=True, inplace=True)

In [8]:
#Testing uniques

all_sites.nunique()

site            5
url_link    15548
author        466
game        12089
company      1822
genre        1248
platform     2398
text        15524
score          77
dtype: int64

In [14]:
#Weird!

all_sites[all_sites.duplicated(subset=['text'])]

,site,url_link,author,game,company,genre,platform,text,score
2644,meristation,https://as.com/meristation/2011/10/31/analisis...,Xevi Costa,Sonic Generations,SEGA,Plataformas,PS3 360 PC,La trayectoria de Sonic estos últimos diez año...,8.5
3257,Vandal,https://vandal.elespanol.com/analisis/ps4/grim...,Ramón Varela,Grim Fandango Remastered - Análisis,Double Fine Productions,Aventura Gráfica Point & Click,PS4 PC PSVITA iPhone Android Switch,Es difícil oponerse a las remasterizaciones cu...,8.5
4648,meristation,https://as.com/meristation/2019/09/10/analisis...,Salva Fernàndez\nsalbaFR,PES 2020,Konami,Deportes,XBO PC PS4,"El fútbol levanta pasiones, y lo hace también ...",8.5
4886,meristation,https://as.com/meristation/2011/04/19/analisis...,Javi Andrés,Portal 2,None,"Acción, First Person Shooter",PC 360 PS3,"The Orange Box, que salió a la venta en octubr...",9.5
4927,meristation,https://as.com/meristation/2011/11/08/analisis...,Javi Andrés,Call of Duty: Modern Warfare 3,None,"Acción, First Person Shooter",360 PS3 PC,"Tan criticado como rompedor en ventas, el fenó...",7.0
6366,revogamers,https://www.revogamers.net/analisis-w/lego-nin...,David Gómez,La LEGO Ninjago Película El Videojuego,None,None,Nintendo Switch,La saga LEGO es una de las más prolíficas que ...,6.5
7069,Vandal,https://vandal.elespanol.com/analisis/ps4/dark...,Sergio Martín,Darksiders: Warmastered Edition - Análisis,Nordic Games,Action-RPG Hack and Slash,PS4 Xbox One PC Wii U Switch,Sin ser uno de los juegos más esperados del mo...,8.3
8896,meristation,https://as.com/meristation/2014/04/10/analisis...,Javi Andrés,LEGO: The Hobbit,None,"Acción, Aventura",PS4 OSX PS3 360 XBO PC WiiU PSV 3DS,Traveller's Tales ha reinterpretado bajo el se...,8.0
9049,meristation,https://as.com/meristation/2011/09/26/analisis...,Javi Andrés,FIFA 12,EA Sports,Deportes,PS3 360 PC,Después de estudiar y resumir los poderes y de...,9.0
9088,meristation,https://as.com/meristation/2009/09/14/analisis...,Ramón Méndez,Section 8,SouthPeak Games,"Acción, First Person Shooter",PS3 PC 360,No se puede decir que Section 8 sea el juego m...,7.0


## 3. Data Standarize 

## 4. Data export

In [15]:
#all_sites.to_csv('../data/all_sites.csv', index=False)

<div style="padding-top: 25px; float: right">
    <div>    
        <i>&nbsp;&nbsp;© Copyright by</i>
    </div>
    <div>
        <a href="https://whiteboxml.com">
            <img src="https://whiteboxml.com/static/img/logo/black_bg_white.svg" width="125">
        </a>
    </div>
</div>